In [ ]:
%matplotlib widget

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from python.fluorophores import FlStatic
from python.psfs import PsfVectorial
from python.estimators import est_donut2d
from python.estimators import est_quad2Diter
from python.estimators import est_quadraticdirect1D
from python.estimators import est_quadraticdirectnobg1D
from python.estimators import est_quadraticdirectbg1D
from python.estimators import backgroundsubtractor
from python.simulators import Simulator
from python.tools import imx

In [ ]:
psf_vec = PsfVectorial()
psf_vec.setpinhole(AU=1)

In [ ]:
fl = FlStatic()
fl.brightness = 10000  # very bright to look at bias
fl.pos = [20, 0, 0]
sim = Simulator(fluorophores=fl)

In [ ]:
numberOfLocalizations = 1000
L = 75
orbitpoints = 4
laserpower = 100
xcoords = np.arange(0,L*0.75,2)
probecenter = True
sim.definePattern("donut", psf_vec, 
                  phasemask="vortex", 
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter,
                  orbitL=L,
                  laserpower=laserpower)

## No background, simple estimator

In [ ]:
ax1v = "pos" 
sim.defineComponent("estdonut", "estimator", est_donut2d, parameters=[sim.patterns["donut"].pos, L, 360], dim=(0,1))
seq=["donut", "estdonut"]
psf_vec.zerooffset = 0

fig = plt.figure()
ax = fig.add_subplot(121)
statout = sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                       display=True, ax1=ax1v, clearfigure=True,
                       tag="simple est")

In [ ]:
# iterative
sim.defineComponent("estiter", "estimator", est_quad2Diter, parameters=[L,probecenter,30], dim=(0,1))
seq = ["donut", "estiter"]
statout = sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                       display=True, ax1=ax1v, clearfigure=False, 
                       tag="iterative est")

In [ ]:
# no background, direct estimator
sim.defineComponent("estdirect", "estimator", est_quadraticdirect1D, parameters=[L], dim=0)
seq = ["donut", "estdirect"]
statout=sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                     display=True, ax1=ax1v, clearfigure=False, 
                     tag="direct lsq", title="comparison of estimators")

In [ ]:
sim.defineComponent("estimator2p", "estimator", est_quadraticdirectnobg1D, parameters=[L,probecenter], dim=0)
seq = ["donut", "estimator2p"]
statout=sim.scan_fov(seq,xcoords, maxlocs=numberOfLocalizations, 
                     display=True, ax1=ax1v, clearfigure=False, 
                     tag="direct eq")

In [ ]:
ax = plt.gca() 
if ax1v=="pos":
    ax.plot([0, L],[0, L],'k--')
ax.set_ylim(bottom=0)

## Explore impact of background on estimator

In [ ]:
# no background
fl.brightness = 1000
fl.pos = [30, 30, 0]
sim.background = 0
sim.defineComponent("estdonut", "estimator",est_quad2Diter, parameters=[L,probecenter], dim=(0,1))
seq = ["donut", "estdonut"]
psf_vec.zerooffset=0

ax = plt.subplot(122)
statout=sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                     display=True, ax1=ax1v, tag="no bg")

In [ ]:
# background, 
sim.background = 50
statout = sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                       display=True, ax1=ax1v, clearfigure=False, tag="bg")

In [ ]:
# background subtracted,
sim.background_estimated=sim.background*laserpower  # in general, the GT background is not known but needs to be calibrated 

sim.defineComponent("bg", "background", backgroundsubtractor, parameters=["background_estimated"])
seq = ["donut", "bg", "estdonut"]
psf_vec.zerooffset = 0
statout = sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                       display=True, ax1=ax1v, clearfigure=False,
                       tag="bg est", title="estimator bias from background")

In [ ]:
sim.defineComponent("estimatorbg", "estimator", est_quadraticdirectbg1D, parameters=[L], dim=0)
seq = ["donut", "estimatorbg"]
statout=sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                     display=True, ax1=ax1v, clearfigure=False,
                     tag="bg fit par", title="estimator bias from background")

In [ ]:
if ax1v=="pos":
    ax.plot([0, L],[0, L],'k--')